In [34]:
import pandas as pd
import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis.gensim

C:\Users\Piyush\Anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


In [2]:
data = pd.read_pickle('webhose_cat.pkl')

In [3]:
data.head()

,crawled,language,text,title,url
0,2018-01-30T18:28:45.012+02:00,english,Avery Dennison's (AVY) Q4 results are likely t...,IRobot downgraded to neutral from buy at Sidot...,http://omgili.com/ri/.wHSUbtEfZQRfU.5KUm1RkeXy...
1,2018-01-30T18:29:07.001+02:00,french,"1m95, c’est trop grand. Et sa stature, Bertran...","""Bertrand Zibi Abeghe, encore prisonnier, et t...",http://omgili.com/ri/.wHSUbtEfZTpzFtnXyQJIwJ.j...
2,2018-01-30T18:29:40.000+02:00,english,Tuggers and Topper Industrial Carts Help Trans...,Tuggers and Topper Industrial Carts Help Trans...,http://omgili.com/ri/jHIAmI4hxg.zDiulpymXqU_n4...
3,2018-01-30T18:30:05.007+02:00,english,Currently adding the following games:\n100 (by...,,http://omgili.com/ri/.0rSU5LtMgyggHgoOVy9TMDWT...
4,2018-01-30T18:30:05.013+02:00,english,Quote: : » Currently adding the following game...,,http://omgili.com/ri/.0rSU5LtMgyggHgoOVy9TMDWT...


Removing non English news articles

In [5]:
data = data[data['language'] == 'english']

In [63]:
articles = data.text.tolist()

Cleaning data by removing stopwords, punctuations and lemmatization

In [64]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [65]:
cleaned_articles = list()
for article in articles:
    
    cleaned_article = list()
    
    for word in article.split():
        if (word in stop):
            #ignore the word
            continue
        
        word = word.lower()
        word =  ''.join([ char for char in word if char not in exclude])
        word = lemma.lemmatize(word)
        if len(word) == 0:
            #Ignore whitespace
            continue
            
        cleaned_article.append(word)
        
    cleaned_articles.append(cleaned_article)

In [66]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(cleaned_articles)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(article) for article in cleaned_articles]

In [67]:
%time ldamodel = LdaMulticore(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50) #3 topics

Wall time: 1min 18s


In [68]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=True, mds='mmds')
pyLDAvis.display(lda_display)

C:\Users\Piyush\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


With n = 3, we see following topics,

1. Something related to China and repatriation
2. New Caterpillar Inc machines in the last quarter of 2018
3. Seems to be a combination of multiple topics. We can see Amazon Seattle, Caterpillar and University town as dominating words.

There does seem to be more than 3 topics

In [71]:
%time ldamodel = LdaMulticore(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50) #5 topics

Wall time: 1min 17s


In [72]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=True, mds='mmds')
pyLDAvis.display(lda_display)

C:\Users\Piyush\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


With n = 5, we see following topics,

1. Impact of taxes on Chinese manufacturing
2. Amazon opening a new office in Seattle in Jan 2018
3. Census of median income, rent, housing prices in different cities, university towns, states
4. Caterpillar Inc stock performance in Jan based on companies performance in Oct, Nov and Dec
5. List of products offered by Caterpillar Inc.

n = 5 seems to perform better than n = 3

In [73]:
%time ldamodel = LdaMulticore(doc_term_matrix, num_topics=8, id2word = dictionary, passes=50) #8 topics

Wall time: 1min 14s


In [74]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=True, mds='mmds')
pyLDAvis.display(lda_display)

C:\Users\Piyush\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


With n = 8, we see following topics,

1. Seems like a combination of multiple topics, such as taxes, university, and Caterpillar
2. Amazon opening a new office in Seattle in Jan 2018 on a Monday
3. Turbocharger and IOT manufacturing report
4. List of products offered by Caterpillar Inc.
5. Caterpillar'snew equipment health monitoring system
6. Neartermp prediction of some stock
7. Seems to be describing some new construction. Probably Amazon's new office
8. Fed seems to be doing some fraud investigation at Caterpillar

###### I think the value of N should be 5, N = 3 gives us less information and N = 8 gives us vague information about multiple topics